In [1]:

import os

%matplotlib inline
import sys
import glob
import rdkit
from rdkit import Chem
from rdkit.Chem import Crippen
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.AllChem import  GetMorganFingerprintAsBitVect, GetErGFingerprint
import matplotlib.pyplot as plt
plt.ion()
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer,
    plot_confusion_matrix,
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from imblearn import under_sampling, over_sampling

In [2]:
df_indexed=pd.read_csv('indexed_final_protacpedia.csv')
df=pd.read_csv('all_experimental_scores.csv')

In [3]:
df_indexed

,pp_plus_linkers_amber_renumbered_mol,mol,Copy of mol,PROTACDB ID,PROTAC SMILES,Active/Inactive,Best PROTAC,Cells,cLogP,Comments,...,Tested A Non Binding E3 Control,Tested Competition With Ligand,Tested Engagement In Cells,Tested Proteaseome Inhibitor,Time,TPSA,ligand_pdb_copy,ligand_pdb,new_name,identifier
0,S(CCC(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C1N(C(=O)C...,NC1=NC2=C(N=C[N]2[C@H]3O[C@@H](CO[P]([O-])(=O)...,P(=O)(OC[C@H]1[C@H](O)[C@H](O)[C@@H](n2c3N=C(N...,1041,P(=O)(OC[C@H]1[C@H](O)[C@H](O)[C@@H](n2c3N=C(N...,Inactive,No,MDA-MB-231; K562,-1.64430,NaN,...,No,No,No,No,24 72,296.25000,1EJ1,1EJ1,1EJ1_1,1ej1_1.pdb
1,S(CCC(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C1N(C(=O)C...,NC1=NC2=C(N=C[N]2[C@H]3O[C@@H](CO[P](O)(=O)O[P...,P(=O)(OP(=O)(O)NCCOCCOCCOCCC(=O)Nc1c2c(C(=O)N(...,1050,P(=O)(OP(=O)(O)NCCOCCOCCOCCC(=O)Nc1c2c(C(=O)N(...,Inactive,No,MDA-MB-231; K562,-2.01580,NaN,...,No,No,No,No,24 72,367.64001,1EJ1,1EJ1,1EJ1_10,1ej1_10.pdb
2,S(CCC(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C1N(C(=O)C...,NC1=NC2=C(N=C[N]2[C@H]3O[C@@H](CO[P](O)(=O)O[P...,P(=O)(OP(=O)(O)NCCOCCOCCOCCOCCOCCOCCOCCC(=O)Nc...,1051,P(=O)(OP(=O)(O)NCCOCCOCCOCCOCCOCCOCCOCCC(=O)Nc...,Inactive,No,MDA-MB-231; K562,-1.94940,NaN,...,No,No,No,No,24 72,404.56000,1EJ1,1EJ1,1EJ1_11,1ej1_11.pdb
3,S(CCC(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C1N(C(=O)C...,NC1=NC2=C(C(=O)N1)[N+](=C[N]2[C@H]3O[C@@H](CO[...,P(=O)(OP(=O)(O)NCCOCCOCCOCCC(=O)Nc1c2c(C(=O)N(...,1052,P(=O)(OP(=O)(O)NCCOCCOCCOCCC(=O)Nc1c2c(C(=O)N(...,Inactive,No,MDA-MB-231; K562,-1.07500,NaN,...,No,No,No,No,24 72,358.63000,1EJ1,1EJ1,1EJ1_12,1ej1_12.pdb
4,S(CCC(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C1N(C(=O)C...,NC1=NC2=C(C(=O)N1)[N+](=C[N]2[C@@H]3O[C@H](CO[...,P(=O)(OP(=O)(O)NCCOCCOCCOCCOCCOCCOCCOCCC(=O)Nc...,1053,P(=O)(OP(=O)(O)NCCOCCOCCOCCOCCOCCOCCOCCC(=O)Nc...,Inactive,No,MDA-MB-231; K562,-1.00860,NaN,...,No,No,No,No,24 72,395.54999,1EJ1,1EJ1,1EJ1_13,1ej1_13.pdb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,S(CCC(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C1N(C(=O)C...,CC[N]1N=C(C=C1NC2=NC(=NC3=C2C4=C([NH]3)C=C(C(=...,O=C(NCCCc1c2c(C(=O)N(C3C(=O)NC(=O)CC3)C2)ccc1)...,110,O=C(NCCCc1c2c(C(=O)N(C3C(=O)NC(=O)CC3)C2)ccc1)...,Active,No,RS4;11 MOLM-13,5.35254,Complex structure with the ligand was modeled ...,...,No,No,No,No,3,202.25999,5VBP 7NQ8 6QJU,7NQ8,7NQ8_5,7nq8_5.pdb
1232,S(=O)(=O)(Nc1c2c3c(N(C)C(=O)c3ccc2)cc1)c1c(OC)...,COC1=CC=CC=C1[S](=O)(=O)NC2=CC=C3N(C)C(=O)C4=C...,S(=O)(=O)(Nc1c2c3c(N(C)C(=O)c3cc(CCC(=O)NCCNc3...,727,S(=O)(=O)(Nc1c2c3c(N(C)C(=O)c3cc(CCC(=O)NCCNc3...,Inactive,No,MV4-11 Molm-13,2.80130,EC50 of ligand is 0.370 uM for MV4- 11 3.369 ...,...,No,No,No,No,1 2 4 6 16 24,200.39000,6C7R 7NQ8,7NQ8,7NQ8_6,7nq8_6.pdb
1233,S(=O)(=O)(Nc1c2c3c(N(C)C(=O)c3ccc2)cc1)c1c(OC)...,COC1=CC=CC=C1[S](=O)(=O)NC2=CC=C3N(C)C(=O)C4=C...,S(=O)(=O)(Nc1c2c3c(N(C)C(=O)c3cc(CCC(=O)NCCCNc...,728,S(=O)(=O)(Nc1c2c3c(N(C)C(=O)c3cc(CCC(=O)NCCCNc...,Active,No,MV4-11 Molm-13,3.19140,EC50 of ligand is 0.370 uM for MV4- 11 3.369 ...,...,No,No,No,No,1 2 4 6 16 24,200.39000,6C7R 7NQ8,7NQ8,7NQ8_7,7nq8_7.pdb
1234,S(=O)(=O)(Nc1c2c3c(N(C)C(=O)c3ccc2)cc1)c1c(OC)...,COC1=CC=CC=C1[S](=O)(=O)NC2=CC=C3N(C)C(=O)C4=C...,S(=O)(=O)(Nc1c2c3c(N(C)C(=O)c3cc(CCC(=O)NCCCCN...,729,S(=O)(=O)(Nc1c2c3c(N(C)C(=O)c3cc(CCC(=O)NCCCCN...,Active,Yes,Big sellection of cacer cell lines,3.58150,EC50 of ligand is 0.370 uM for MV4- 11 3.369 ...,...,No,No,No,No,1 2 4 6 16 24,200.39000,6C7R 7NQ8,7NQ8,7NQ8_8,7nq8_8.pdb


In [4]:
from Bio.PDB.ResidueDepth import ResidueDepth
from Bio.PDB.PDBParser import PDBParser
parser = PDBParser()

files=[glob.glob('pois/*pdb')][0]
for index, row in df_indexed.iterrows():
    #f=[glob.glob(f"pois/poi_{row['ligand_pdb'].lower()}*pdb")][0][0]
    structure = parser.get_structure(f"{row['ligand_pdb'].lower()}", f"pois/poi_{row['new_name'].lower()}_docked_rec.pdb")
    print(structure)
    model = structure[0]
    rd = ResidueDepth(model)
    print(rd)


/home/hmslati/.local/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:421: PDBConstructionWarning: PDBConstructionException: Atom C defined twice in residue <Residue UNL het=H_UNL resseq=192 icode= > at line 3086.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  PDBConstructionWarning,
/home/hmslati/.local/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:421: PDBConstructionWarning: PDBConstructionException: Atom C defined twice in residue <Residue UNL het=H_UNL resseq=192 icode= > at line 3087.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  PDBConstructionWarning,
/home/hmslati/.local/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:421: PDBConstructionWarning: PDBConstructionException: Atom C defined twice in residue <Residue UNL het=H_UNL resseq=192 icode= > at line 3088.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  PDBConstructionWarning,
/home/hmslati/.local/lib/python3.6/site-

<Structure id=1ej1>


RuntimeError: Failed to generate surface file using command:
msms -probe_radius 1.5 -if /tmp/tmp1xv2axqe -of /tmp/tmpc6v2xui4 > /tmp/tmp3mq_qro9